# Canonical pipeline plate acquisition script
__Keith Cheveralls__<br>
__November 2019__

This notebook sets up and executes a pipeline plate acquisition. There is very little user input required; the only cell that should be edited is the one in which the directory to which the images and logfiles should be saved. 

__Before running the cells in this notebook, please be sure that__
1. A position list generated by the 'HCS Site Generator' plugin is loaded in MicroManager
2. The position list contains interpolated FocusDrive positions (generated in the `stage-position-interpolation.ipynb` notebook)
3. The mm2python bridge has been created and the monitor has been started

Please note that if any Python errors occur when running any of the cells in this notebook, something unexpected has happened and Keith should be contacted. 

In [ ]:
import os
import re
import sys
import git
import json
import skimage
import sklearn
import tifffile
import logging
import datetime
import numpy as np
import pandas as pd
from py4j.java_gateway import JavaGateway

sys.path.insert(0, '..')
from dragonfly_automation import utils
from dragonfly_automation.gateway import gateway_utils
from dragonfly_automation.fov_models import PipelineFOVScorer
from dragonfly_automation.acquisitions.pipeline_plate_acquisition import PipelinePlateAcquisition

### Set the location of the directory to which to save the images

__Note: this directory must not already exist!__

In [ ]:
experiment_dir = os.path.join('D:', 'PR', 'dragonfly-automation-tests',' ML0000_20191003')

### Load the trained FOV-scoring class

This cells takes a few seconds to run and prints some output that can be ignored.

In [ ]:
fov_scorer = PipelineFOVScorer(mode='prediction')
fov_scorer.load('../models/2019-10-08/')
fov_scorer.train()
fov_scorer.validate()

### Initialize the acquisition

None of the parameters here should need to be changed.

In [ ]:
aq = PipelinePlateAcquisition(
    experiment_dir, 
    fov_scorer, 
    env='prod', 
    verbose=True,
    test_mode=None)

### Initial microscope configuration and setup
This cell should run essentially instantly.

In [ ]:
aq.setup()

### Start the acquisition

This cell runs the acquisition itself, which generally takes 4-12 hours to complete. If everything is working, log messages should start to appear here within a few seconds. They should be self-explanatory. Note that some log messages may be error messages, which is normal-ish and does not indicate a problem. However, if a real uncaught Python exception occurs (that is, lots of unreadable and color-coded text suddenly appears and the log message stop appearing), then something unexpected and bad has happened. 

Note that AFC timeout errors will __not__ cause the script to crash but will generate a logged error message.

In [ ]:
aq.run()